In [1]:
# Imports
import rasterio
from rasterio.windows import Window
import time
from skimage.io import imread
from PIL import Image
import numpy as np
import os

In [2]:
# GeoTIFF from NASA src path

topoPathA1 = 'data/topography_10800x10800_A1.tif'
topoPathA2 = 'data/topography_10800x10800_A2.tif'
topoPathB1 = 'data/topography_10800x10800_B1.tif'
topoPathB2 = 'data/topography_10800x10800_B2.tif'
topoPathC1 = 'data/topography_10800x10800_C1.tif'
topoPathC2 = 'data/topography_10800x10800_C2.tif'
topoPathD1 = 'data/topography_10800x10800_D1.tif'
topoPathD2 = 'data/topography_10800x10800_D2.tif'

topoArrayPath = []
topoArrayPath.append(topoPathA1)
topoArrayPath.append(topoPathA2)
topoArrayPath.append(topoPathB1)
topoArrayPath.append(topoPathB2)
topoArrayPath.append(topoPathC1)
topoArrayPath.append(topoPathC2)
topoArrayPath.append(topoPathD1)
topoArrayPath.append(topoPathD2)

# Size of tiles
xSize, ySize = 256, 256

In [3]:
# returns True if we should keep heightmap tilek, otherwise False to discard it
# counts how much black in heightmap (to discard only water areas)
def comparator(heightmap_chunk):
    black_ratio = ((heightmap_chunk==0).sum()*1.0)/float(np.prod(heightmap_chunk.shape))
    if black_ratio > 0.9:
        return False
    return True

In [4]:
# Cut geotiff images and save to folder, remove images where 90% area is water

iterator = 0

for srcTifPath in topoArrayPath:
    t0 = time.time()
    insideIterator = 0
    with rasterio.open(srcTifPath) as src:  
        for y in range(0, src.width, 256):
            for x in range(0, src.height, 256):
                iterator += 1
                insideIterator += 1
                # Create a Window and calculate the transform from the source dataset    
                window = Window(x, y, xSize, ySize)
                transform = src.window_transform(window)

                # Create a new cropped raster to write to
                profile = src.profile
                profile.update({
                    'height': xSize,
                    'width': ySize,
                    'transform': transform})
            
                savePathName = 'cut_tif_data/topography_' + str(iterator) + '.tif'
                with rasterio.open(savePathName, 'w', **profile) as dst:
                    # Read the data from the window and write it to the output raster
                    dst.write(src.read(window=window))
            
                texture = imread(savePathName)
            
                if not comparator(texture):
                    os.remove(savePathName)
                    iterator -= 1
                    insideIterator -= 1
    print(str(time.time()-t0) + 's for ' + srcTifPath + '. Saved ' + str(insideIterator) + ' images')

9.032999038696289s for data/topography_10800x10800_A1.tif. Saved 563 images
8.418999671936035s for data/topography_10800x10800_A2.tif. Saved 352 images
8.775999307632446s for data/topography_10800x10800_B1.tif. Saved 680 images
8.899997234344482s for data/topography_10800x10800_B2.tif. Saved 704 images
9.523000001907349s for data/topography_10800x10800_C1.tif. Saved 1212 images
9.427156925201416s for data/topography_10800x10800_C2.tif. Saved 700 images
9.544564008712769s for data/topography_10800x10800_D1.tif. Saved 742 images
9.003003120422363s for data/topography_10800x10800_D2.tif. Saved 733 images
